In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from keras.models import model_from_json
import os
from keras.models import load_model

Using TensorFlow backend.


In [2]:
dataframes = pandas.read_csv("../data/files/testdefinitivo.csv", delimiter=',',error_bad_lines=False,compression='gzip')
dataframes.head(3)


,floor,id,lat-lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,...,comedor,sum,patio,palier,rooms1,rooms2,surface_total_in_m21,surface_total_in_m22,surface_covered_in_m21,surface_covered_in_m22
0,0.0,3632,6287.862607,11.987854,-6.241196,-0.883849,0.071429,-0.000000,0.037131,0.035609,...,1.0,0.0,0.0,0.0,0.2,0.333333,0.035443,0.035533,0.036458,0.036458
1,0.0,3633,-6472.820151,28.533506,-1.614436,-0.883849,NaN,2.538785,NaN,NaN,...,1.0,1.0,0.0,0.0,0.0,0.000000,-0.005063,-0.005076,-0.005208,-0.005208
2,0.0,2263404,3689.986230,42.399812,-1.984977,-0.883849,0.035714,-0.000000,0.020313,0.021783,...,0.0,1.0,0.0,0.0,0.2,0.333333,0.022785,0.020305,0.020833,0.020833


In [3]:
dataframes.columns.tolist()
test=dataframes[['surface_covered_in_m2', 'place_name', 'place_with_parent_names', 'surface_total_in_m2', 'hidromasaje', 'laundry', 'parquet', 'pileta', 'cochera', 'jacuzzi', 'hall', 'vista', 'educacion', 'solarium', 'parrilla', 'antigua', 'esquina', 'patio', 'surface_total_in_m21']]

test.fillna(test.mean(),inplace= True)
test.head(10)

/usr/lib/python2.7/dist-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,surface_covered_in_m2,place_name,place_with_parent_names,surface_total_in_m2,hidromasaje,laundry,parquet,pileta,cochera,jacuzzi,hall,vista,educacion,solarium,parrilla,antigua,esquina,patio,surface_total_in_m21
0,0.037131,11.987854,-6.241196,0.035609,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035443
1,0.044427,28.533506,-1.614436,0.072466,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005063
2,0.020313,42.399812,-1.984977,0.021783,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.022785
3,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
4,0.021875,-42.517241,-67.410441,0.020770,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.020253
5,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
6,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
7,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
8,0.030208,-3.028289,-72.037559,0.028875,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.027848
9,0.019792,42.980368,49.704325,0.018744,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.017722


In [4]:
xtest = test.values
 
xtest=xtest[:,0:19]

In [5]:
xtest 

array([[  3.71308855e-02,   1.19878535e+01,  -6.24119595e+00, ...,
          0.00000000e+00,   0.00000000e+00,   3.54430380e-02],
       [  4.44267460e-02,   2.85335057e+01,  -1.61443632e+00, ...,
          0.00000000e+00,   0.00000000e+00,  -5.06329114e-03],
       [  2.03125000e-02,   4.23998116e+01,  -1.98497737e+00, ...,
          0.00000000e+00,   0.00000000e+00,   2.27848101e-02],
       ..., 
       [  6.56250000e-02,   1.92807428e+01,   3.29115211e+01, ...,
          0.00000000e+00,   0.00000000e+00,   1.06329114e-01],
       [  2.03125000e-02,  -2.92739767e+01,   1.64150838e+01, ...,
          0.00000000e+00,   0.00000000e+00,   2.53164557e-02],
       [  6.30208333e-02,   1.56192406e+01,  -4.61370526e+00, ...,
          1.00000000e+00,   0.00000000e+00,   1.79746835e-01]])

In [6]:
dataframe = pandas.read_csv("../data/files/training_set_definitivo(1).csv", delimiter=',',error_bad_lines=False,compression='gzip')
dataframe.head(5)

,floor,id,lat-lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,...,comedor,sum,patio,palier,rooms1,rooms2,surface_total_in_m21,surface_total_in_m22,surface_covered_in_m21,surface_covered_in_m22
0,0.000000,ad916bad59cdaf3c24c2d594547aa6067c4d6289,6221.739792,-11.833789,-4.875500,0.689885,0.035714,-0.0,0.016146,0.022796,...,1.0,0.0,1.0,0.0,0.2,0.333333,0.022785,0.020305,0.015625,0.015625
1,0.063830,61cb773b63b911ad2ea5c823187d7ba1c7f3cadf,-528.644641,-11.833789,-4.875500,0.278800,0.035714,-0.0,0.023958,0.022796,...,1.0,0.0,0.0,0.0,0.2,0.333333,0.022785,0.020305,0.023438,0.020833
2,0.000000,0955112ee519f7f80496e495187595fd91177b3c,-2140.051350,-29.273977,16.415084,0.278800,0.000000,-0.0,0.016146,0.017730,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.017722,0.015228,0.015625,0.015625
3,0.000000,e421fd04e1d202cad203372bd3ea7fd448155553,-2140.051350,-29.273977,16.415084,0.278800,0.035714,-0.0,0.026562,0.027862,...,0.0,0.0,0.0,0.0,0.2,0.333333,0.027848,0.025381,0.026042,0.026042
4,0.021277,ae0d3e36fac1fa60148d7f747b0d8c7c65f2db78,6059.859072,42.399812,-1.984977,0.278800,0.071429,-0.0,0.010937,0.020263,...,0.0,0.0,0.0,0.0,0.2,0.333333,0.020253,0.020305,0.010417,0.010417


In [7]:

entrenamiento=dataframe[['surface_covered_in_m2', 'place_name', 'place_with_parent_names', 'surface_total_in_m2', 'hidromasaje', 'laundry', 'parquet', 'pileta', 'cochera', 'jacuzzi', 'hall', 'vista', 'educacion', 'solarium', 'parrilla', 'antigua', 'esquina', 'patio', 'surface_total_in_m21','price_aprox_usd']]

entrenamiento.fillna(entrenamiento.mean(),inplace= True)

entrenamiento.head(5)

,surface_covered_in_m2,place_name,place_with_parent_names,surface_total_in_m2,hidromasaje,laundry,parquet,pileta,cochera,jacuzzi,hall,vista,educacion,solarium,parrilla,antigua,esquina,patio,surface_total_in_m21,price_aprox_usd
0,0.016146,-11.833789,-4.875500,0.022796,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.022785,62000.0
1,0.023958,-11.833789,-4.875500,0.022796,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022785,72000.0
2,0.016146,-29.273977,16.415084,0.017730,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.017722,138000.0
3,0.026562,-29.273977,16.415084,0.027862,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.027848,195000.0
4,0.010937,42.399812,-1.984977,0.020263,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.020253,111700.0


In [8]:
dataset = entrenamiento.values
 
X = dataset[0:,0:19]
Y = dataset[0:,19:20]

In [9]:
X[:4]
 

array([[  1.61458333e-02,  -1.18337893e+01,  -4.87550036e+00,
          2.27963526e-02,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          2.27848101e-02],
       [  2.39583333e-02,  -1.18337893e+01,  -4.87550036e+00,
          2.27963526e-02,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.27848101e-02],
       [  1.61458333e-02,  -2.92739767e+01,   1.64150838e+01,
          1.77304965e-02,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.0000

In [10]:
Y[:4]

array([[  62000.],
       [  72000.],
       [ 138000.],
       [ 195000.]])

In [22]:
 
# define base model
def baseline_model():

    # create model
    model = Sequential()
    model.add(Dense(300, input_dim=19, init="normal", activation='relu'))#cantidad de entradas es el 4,inputdim
    #model.add(Dense(150, init='normal', activation='relu'))
    #model.add(Dense(150, init='normal', activation='relu'))
    
     
    model.add(Dense(150, init='normal', activation='relu'))
    model.add(Dense(150, init='normal', activation='relu')) 
    model.add(Dense(150, init='normal', activation='relu'))
    model.add(Dense(150, init='normal', activation='relu'))
    model.add(Dense(1, init="normal"))#salida
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    
    return model

In [23]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
kfold = KFold(n_splits=2, random_state=seed) #splits 5 o 10 

estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=10, batch_size=32, verbose=2)

results = cross_val_score(estimator, X, Y, cv=kfold)

print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
estimator.fit(X, Y)

print("Entrenando modelo... OK")

print("")
print("Salvando el modelo en archivo")

prediction = estimator.predict(xtest)
 



Epoch 1/10
11s - loss: 75997764025.0369
Epoch 2/10
13s - loss: 56215696250.9388
Epoch 3/10
10s - loss: 52809715762.6331
Epoch 4/10
10s - loss: 50361058164.4605
Epoch 5/10
10s - loss: 47757374683.3251
Epoch 6/10
12s - loss: 46933536653.4361
Epoch 7/10
13s - loss: 45114079810.6158
Epoch 8/10
11s - loss: 43423764010.4926
Epoch 9/10
16s - loss: 42025968661.2250
Epoch 10/10
13s - loss: 41590462217.9093
Epoch 1/10
14s - loss: 91068677806.4457
Epoch 2/10
10s - loss: 69575253477.7032
Epoch 3/10
10s - loss: 66527442529.3878
Epoch 4/10
10s - loss: 64319808877.5986
Epoch 5/10
10s - loss: 62046898395.0693
Epoch 6/10
10s - loss: 59813009768.7399
Epoch 7/10
10s - loss: 58146609420.5091
Epoch 8/10
10s - loss: 56437767303.4479
Epoch 9/10
10s - loss: 54131766984.0180
Epoch 10/10
10s - loss: 52547158064.9283
Results: 54080171512.66 (1846508591.58) MSE
Epoch 1/10
20s - loss: 73479804808.0659
Epoch 2/10
20s - loss: 59336340656.1931
Epoch 3/10
21s - loss: 54589781555.4429
Epoch 4/10
20s - loss: 49325476096

In [24]:
estimator.model.save('model_train.h5')

In [25]:
# serialize model to JSON
 
#json_model = estimator.model.to_json()
 
#estimator.model.save_weights('model_weights.h5', overwrite=True) 




In [26]:
for x in prediction:
     print(x)
  

483089.0
264122.0
120470.0
150459.0
158282.0
150459.0
150459.0
150459.0
225376.0
105688.0
105688.0
105688.0
425183.0
658811.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
165101.0
165985.0
165985.0
165985.0
165985.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
154466.0
127345.0
245453.0
425183.0
425183.0
126673.0
187089.0
152682.0
104471.0
234804.0
143229.0
298880.0
228337.0
613523.0
158774.0
429892.0
109765.0
187974.0
149063.0
112330.0
571824.0
93366.6
125997.0
258799.0
144166.0
101199.0
125597.0
142155.0
125597.0
125052.0
126816.0
234441.0
126931.0
126931.0
126816.0
126816.0
126816.0
126816.0
458134.0
126816.0
137365.0
458134.0
155107.0
121329.0
191016.0
116182.0
341304.0
138556.0
128470.0
443217.0
106182.0
330845.0
203132.0
200333.0
200333.0
521146.0
250241.0
371466.0
95911.4
290650.0
316713.0
170190.0
308324.0
308324.0
545486.0
148383.0
171856.0
155110.0
228328.0
106736.0
107636.0
110

In [27]:
prediction.size


14166

In [28]:
ids=dataframes['id'].values
precios = pandas.DataFrame(ids ,columns={'id'})
precios.head(5)

,id
0,3632
1,3633
2,2263404
3,2263405
4,2263406


In [29]:


precios['price_usd']=prediction

In [30]:
precios

,id,price_usd
0,3632,4.830890e+05
1,3633,2.641223e+05
2,2263404,1.204699e+05
3,2263405,1.504588e+05
4,2263406,1.582823e+05
5,2263407,1.504588e+05
6,2263408,1.504588e+05
7,2263409,1.504588e+05
8,2263410,2.253760e+05
9,2263411,1.056877e+05


In [31]:
precios

,id,price_usd
0,3632,4.830890e+05
1,3633,2.641223e+05
2,2263404,1.204699e+05
3,2263405,1.504588e+05
4,2263406,1.582823e+05
5,2263407,1.504588e+05
6,2263408,1.504588e+05
7,2263409,1.504588e+05
8,2263410,2.253760e+05
9,2263411,1.056877e+05


In [32]:
precios.to_csv("../data/files/resultados4.csv",index=False)